In [1]:
%matplotlib inline
import nibabel as nib
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import subprocess
import sys

In [2]:
#import sys
#sys.path.append('/Users/evaalonsoortiz/Documents/python/susceptibility-to-fieldmap-fft/')
#from functions import compute_fieldmap

In [2]:
# Select the path to the folder so that it can always address data folder correctly
home_path = "C:/Users/User/msc_project/sus-optimization"

### Hash where I had the fft bug aka template FM HZ created from: 
*d9f785b082fb145d547ff03ae53f23f1564ccc38*888

In [3]:
# Above method uses the "main" simulation, so we need to load the latest
!git clone https://github.com/shimming-toolbox/susceptibility-to-fieldmap-fft.git
%cd susceptibility-to-fieldmap-fft
!git pull 
!git checkout mb/custom_pad
!git checkout d9f785b082fb145d547ff03ae53f23f1564ccc38
%pip install .

c:\Users\User\msc_project\sus-optimization\susceptibility-to-fieldmap-fft


Cloning into 'susceptibility-to-fieldmap-fft'...
c:\Users\User\miniconda3\envs\mrsim\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Already up to date.
branch 'mb/custom_pad' set up to track 'origin/mb/custom_pad'.


Switched to a new branch 'mb/custom_pad'
Note: switching to 'd9f785b082fb145d547ff03ae53f23f1564ccc38'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by switching back to a branch.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -c with the switch command. Example:

  git switch -c <new-branch-name>

Or undo this operation with:

  git switch -

Turn off this advice by setting config variable advice.detachedHead to false

HEAD is now at d9f785b Fix bug


Processing c:\users\user\msc_project\sus-optimization\susceptibility-to-fieldmap-fft
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for susceptibility-to-fieldmap-fft: filename=susceptibility_to_fieldmap_fft-0.0.0-py3-none-any.whl size=23545 sha256=c0914fa0f741b67682a41f3aa055a6eba503caa7decedf66ce02e8ef6338f696
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\17\ea\51\22427b7b5d1aced006c3db69245d8a8f88effcb4349a030d61
Successfully built susceptibility-to-fieldmap-fft
  Attempting uninstall: susceptibility-to-fieldmap-fft
    Found existing installation: susceptibility-to-fieldmap-fft 0.0.0
    Uninstalling susceptibility-to-fieldmap-fft-0.0.0:
      Successfully uninstalled susceptibility-to-fieldmap-fft-0.0.0
Note: you may need to restart the kernel to use updated packages.


In [4]:
!git rev-parse HEAD

d9f785b082fb145d547ff03ae53f23f1564ccc38


In [5]:
#Once we confirmed the head of the chi to fbfest we can go to home
%cd {home_path}
!ls

C:\Users\User\msc_project\sus-optimization
README.md
data
evas_code.ipynb
internal_air_only.nii.gz
output
requirements.txt
simulation
soothsayer.ipynb
spinal_cord_voxels_only.nii.gz
sus_optimization.ipynb
susceptibility-to-fieldmap-fft
tissue-to-MRproperty


In [16]:
path_to_chi_to_fm_fft = "C:/Users/User/msc_project/susceptibility-to-fieldmap-fft"
sys.path.append(path_to_chi_to_fm_fft)
#sys.path.append('/Users/evaalonsoortiz/Documents/python/susceptibility-to-fieldmap-fft/')
from functions import compute_fieldmap

In [20]:
#meas_fmap_path = "../data/fm_Hz.nii.gz"
#sim_chi_map_path = "../data/chimap_crop_custom6.nii.gz"
#ROIs_map_path = "../data/mask_crop_gre_fov_body_plus_trachea.nii.gz" # Segmentations

In [6]:
# We need to reate the custom command that creates the fieldmap
def chi_to_fm_ppm(input_file, output_file):

    #Using a custom command for dB0 simulation

    command = [
    "compute_fieldmap",
    "-i", input_file,
    "-o", output_file,
    "-b", "50",
    "-m", "b0SimISMRM"
    ]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)

    # We need to return the data from the fieldmap created

    
    # Check for errors
    if result.returncode != 0:
        print("Error occurred:", result.stderr)

    else:
        print("Fieldmap simulated correctly", result.stdout)
        sim_fm_img = nib.load(output_file)
        sim_fm_data = sim_fm_img.get_fdata()
        return sim_fm_data

## Load variable paths!

In [6]:
# New paths to cropped ISMRM
sim_chi_map_path3 = "data/crop_ISMRM/db0_030_wb_chimap_crop_ISMRM.nii.gz"
segmentations_cropped = "data/crop_ISMRM/db0_030_segmentations_cropped_ISRMRM.nii.gz"
sc_final_mask = "data/crop_ISMRM/sc_seg_c2t7_final_ISMRM.nii.gz"
measured_B0 = "data/crop_ISMRM/B0_cropped_ISMRM.nii.gz"
demodulating_mask_path = "data/crop_ISMRM/sc_seg_dilate2_10.nii.gz"

## Objective function </br>
Minimize the difference of the values on the spinal cord between B0meas and iterativaly changing fieldmaps with different chi values!

In [12]:
def f(x):

    print("Using chi: ",x)
    # load simulated susceptibility map in ppm
    chi_img = nib.load(sim_chi_map_path3)
    chi_data = chi_img.get_fdata()
    
    # load segmentation labels map
    ROIs_img = nib.load(segmentations_cropped)
    ROIs_data = ROIs_img.get_fdata()

    sc_seg_img = nib.load(sc_final_mask)
    sc_seg_data = sc_seg_img.get_fdata()

    demod_mask_img = nib.load(demodulating_mask_path)
    demod_mask_data = demod_mask_img.get_fdata()


    ind_sc = np.where(sc_seg_data == 1)
    # find indicies that have label = 7 or 8, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
    ind_tl = np.where((ROIs_data == 7) | (ROIs_data == 8))
    

    # load simulated field map in Hz
    b0meas_img = nib.load(measured_B0)
    b0meas_data = b0meas_img.get_fdata()

    # assign variable "x" to indicies corresponding to trachea/lungs
    chi_data[ind_tl] = x

    # compute the field distribution in ppm for the chi_data distribution using the compute_bz function from the susceptibility-to-fieldmap-fft package
    #z =  compute_fieldmap.compute_bz(chi_data)
    chi_dist, image_res , affine_matrix = compute_fieldmap.load_sus_dist(sim_chi_map_path3)
    #z = chi_to_fm_ppm(sim_chi_map_path2, output"output/slurp.nii.gz")
    # convert field distribution from ppm to Hz (assuming 3T)
    z = compute_fieldmap.compute_bz(chi_data, image_resolution = image_res , buffer = 50, mode = "b0SimISMRM")
    z = z*3*42.58 # deltaB0 [Hz] = deltaB0 [ppm] * B0 [T] * 1e-6 * 42.58*e6 [Hz/T]

    # Demdoulate Z and b0meas to their own zeros
    # Here we coul check if the voxel is bone or tissue (later on)
    demod_factor_b0meas = np.mean(b0meas_data[demod_mask_data == 1])
    print("Demod factor for B0meas: ", demod_factor_b0meas)
    demod_factor_z = np.mean(z[demod_mask_data == 1])
    print("Demod factor for Simulated: ", demod_factor_z)
    demod_b0_meas = b0meas_data - demod_factor_b0meas
    demod_z = z - demod_factor_z
    difference = np.linalg.norm(demod_b0_meas[ind_sc] - demod_z[ind_sc])
    print("minizing this difference: ", difference)
    return (np.linalg.norm(demod_b0_meas[ind_sc] - demod_z[ind_sc]))

In [18]:
min_chi_tl = optimize.fmin(f, -3)
print("Minimum chi:", min_chi_tl[0])

[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0.5 0.5]
[291 385 943] [0.5 0

In [14]:
bounds = [(-5,0.4)]
min_chi_tl = optimize.minimize(f, -4.2, bounds=bounds)
print("Minimum chi:", min_chi_tl.x[0])
print("Optimization success:", min_chi_tl.success)  # Check if the optimization was successful
print("Message:", min_chi_tl.message) 

Using chi:  [-4.2]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.37221739257666
Using chi:  [-4.19999999]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.37221739325076
Using chi:  [0.4]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.6823132305558
Using chi:  [0.39999999]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.6823132298815
Minimum chi: 0.4
Optimization success: True
Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL


In [10]:
print("Minimum chi:", min_chi_tl.x[0])

Minimum chi: [-0.20583366]


In [16]:
bounds = [(-9.05,0)]
min_chi_tl = optimize.minimize(f, 0.35, bounds=bounds)
print("Minimum chi:", min_chi_tl.x[0])
print("Optimization success:", min_chi_tl.success)  # Check if the optimization was successful
print("Message:", min_chi_tl.message) 

Using chi:  [0.]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.65534837507926
Using chi:  [-1.e-08]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.6553483744052
Minimum chi: 0.0
Optimization success: True
Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL


## With new chi value generate a chimap and corresponding FM

In [17]:
opt_chi = np.floor(min_chi_tl.x[0]  * 100) / 100.0  #min_chi_tl.x[0]  min_chi_tl[0]
opt_chi

0.0

## eXTRA Code - Testing everything
For the new chimaps, there is no spinal cord added but we have the spina cord segmentation

In [19]:
    # load simulated susceptibility map in ppm
chi_img = nib.load(sim_chi_map_path3)
chi_data = chi_img.get_fdata()
    
    # load segmentation labels map
ROIs_img = nib.load(segmentations_cropped)
ROIs_data = ROIs_img.get_fdata()

sc_seg_img = nib.load(sc_final_mask)
sc_seg_data = sc_seg_img.get_fdata()

demod_mask_img = nib.load(demodulating_mask_path)
demod_mask_data = demod_mask_img.get_fdata()


ind_sc = np.where(sc_seg_data == 1)
    # find indicies that have label = 7 or 8, these voxels should correspond to the trachea (t) and lungs (l); ind_tl
ind_tl = np.where((ROIs_data == 7) | (ROIs_data == 8))
    

    # load simulated field map in Hz
b0meas_img = nib.load(measured_B0)
b0meas_data = b0meas_img.get_fdata()

    # assign variable "x" to indicies corresponding to trachea/lungs
chi_data[ind_tl] = -4.2 # x in the loop above

    # compute the field distribution in ppm for the chi_data distribution using the compute_bz function from the susceptibility-to-fieldmap-fft package
    #z =  compute_fieldmap.compute_bz(chi_data)
chi_dist, image_res , affine_matrix = compute_fieldmap.load_sus_dist(sim_chi_map_path3)
    #z = chi_to_fm_ppm(sim_chi_map_path2, output"output/slurp.nii.gz")
    # convert field distribution from ppm to Hz (assuming 3T)
z = compute_fieldmap.compute_bz(chi_data, image_resolution = image_res , buffer = 50, mode = "b0SimISMRM")
z = z*3*42.58 # deltaB0 [Hz] = deltaB0 [ppm] * B0 [T] * 1e-6 * 42.58*e6 [Hz/T]

demod_factor_z = np.mean(z[demod_mask_data == 1])
print("Demod factor for Simulated: ", demod_factor_z)
demod_z = z - demod_factor_z


Demod factor for Simulated:  nan


(191, 285, 843)

In [15]:
# Quickly check if the ind_tl is actually taking the lungs or not
ind_tl = np.where((ROIs_data == 7) | (ROIs_data == 8))

only_internal_air = np.zeros(np.shape(ROIs_data))
only_internal_air[ind_tl] = 1

only_air_img = nib.Nifti1Image(only_internal_air, affine=ROIs_img.affine)
nib.save(only_air_img, "internal_air_only.nii.gz")

### Using subprocess to test if we are using the correct version of compute_bz

In [8]:
# For sure that compute_bz is not returning the same even using the same chi value
# Lets test calling with the custom function
def chi_to_fm_ppm(input_file, output_file):

    #Using a custom command for dB0 simulation

    command = [
    "compute_fieldmap",
    "-i", input_file,
    "-o", output_file,
    "-b", "50",
    "-m", "b0SimISMRM"
    ]

    # Run the command and capture output
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Check for errors
    if result.returncode != 0:
        print("Error occurred:", result.stderr)
    else:
        print("FM created successfully:", result.stdout)

In [9]:
chi_to_fm_ppm(sim_chi_map_path3,"output/please_please_please.nii.gz")

FM created successfully: Start
Susceptibility distribution loaded
Fieldmap simulated
Saving to NIfTI format
End. Runtime: 20.40 seconds



In [36]:
z_img_demod = nib.Nifti1Image(demod_z, affine = chi_img.affine)
nib.save(z_img_demod, "output/z_demoded_img.nii.gz")

In [33]:
sc_voxels_only = np.zeros_like(sc_seg_data)

# Fill the spinal cord voxels
sc_voxels_only[ind_sc] = demod_z[ind_sc]

# Save the new mask as a NIfTI file for visual inspection
masked_img = nib.Nifti1Image(sc_voxels_only, affine=sc_seg_img.affine)
nib.save(masked_img, "spinal_cord_voxels_only.nii.gz")


In [ ]:
bounds = [(-5,0.4)]
min_chi_tl = optimize.minimize(f, -4.2, bounds=bounds)
print("Minimum chi:", min_chi_tl.x[0])
print("Optimization success:", min_chi_tl.success)  # Check if the optimization was successful
print("Message:", min_chi_tl.message) 

Using chi:  [-4.2]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.37221739257666
Using chi:  [-4.19999999]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.37221739325076
Using chi:  [0.4]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.6823132305558
Using chi:  [0.39999999]
Demod factor for B0meas:  132.99112910454772
Demod factor for Simulated:  -298.6823132298815
Minimum chi: 0.4
Optimization success: True
Message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL


In [26]:
# What I want to do is mask this voxels out and see if they are literally the same 
masked_b0meas = np.zeros(np.shape(b0meas_data))
masked_chi_dist = np.zeros(np.shape(chi_data))

masked_b0meas = b0meas_data[sc_seg_data == 1]

masked_b0_img = nib.Nifti1Image(masked_b0meas, affine=b0meas_img.affine)
nib.save(masked_b0_img, "output/masked/masked_B0meas.nii.gz")

masked_chimap = chi_data[ind_sc]

masked_chimap_img = nib.Nifti1Image(masked_chimap, affine=chi_img.affine)
nib.save(masked_chimap_img, "output/masked/mask_chimap.nii.gz")

In [23]:
np.shape(b0meas_data)

(191, 285, 843)

In [ ]:
z_img = nib.nifti1Image(z, affine = chi_img.affine)
nib.save(z_img, "output/z_image_chi.nii.gz")